In [ ]:
import util.util as util
import re
import importlib
import os
import json
#importlib.reload(util.util)

In [ ]:
# TODO: set the path to your result folder
# set the first argument to the base path of the result folder, second argument is th folder name, third argument a mapping file - that tells for apps in multiple sub datasets which app to include

verified_dataset = util.get_verified_dataset("//", "/2023_04_06/", "/verified_dataset/result.json")


In [ ]:
# TODO: set the path to load your data
genDataset = util.loadAllData("/gp_2022/", True)

In [ ]:


def getLocalAddresses(domains):
    result = set()
    for d in domains:
        if util.isLocal(d):
            if '.local' in d:
                #print(d)
                if 'fromui.local' in d:
                    result.add('fromui.local')
            #elif d.startswith('127.') or d.startswith('0.'):
            #    continue
            else:
                result.add(d)

    return result

def getAllLocalAddresses(app):
    result = set()
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.amqp)))
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.coap)))
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.endpoints)))
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.mqtt)))
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.xmpp)))
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.udp)))
    result.update(getLocalAddresses(util.getUniqueLocalCommunicationJson(app.webview)))

    return result


In [ ]:
def hasUPNP(jsonData):
    if 'ValuePoints' in jsonData:
        for vp in jsonData['ValuePoints']:
            if 'UsesPotentiallyUPnP' in vp:
                print(jsonData['pname'])
                return True

    return False

def isAppUsingUPNP(app):
    return hasUPNP(app.amqp) or hasUPNP(app.coap) or hasUPNP(app.endpoints) or hasUPNP(app.mqtt)  or hasUPNP(app.xmpp)

#addresses = re.compile("224\.0\.0\.|224\.0\.1\.|224\.0\.2\.| 224\.1\.| 224\.3\.| 224\.4\.| 225\.| 226\.| 227\.| 228\.| 229\.| 230\.| 231\.| 232\.|233\.| 234\.| 235\.|236\.| 237\.| 238\. | 239\.| ffx0:| ffx1:| ffx2:|ffx3:|ffx4:|ffx5:|ffx6:|ffx7:|ffx8:|ffx9:|ffxa:|ffxb:|ffxc:|ffxd:|ffxe:| ff02:| ff0x:")
def doesAppContainMulticast(app):
    details = ['255.255.255.255','224.0.0.','224.0.1.','224.0.2.', '224.1.', '224.3.', '224.4.', '225.', '226.', '227.', '228.', '229.', '230.', '231.', '232.','233.', '234.', '235.','236.', '237.', '238.' , '239.', 'ffx0:', 'ffx1:', 'ffx2:','ffx3:','ffx4:','ffx5:','ffx6:','ffx7:','ffx8:','ffx9:','ffxa:','ffxb:','ffxc:','ffxd:','ffxe:', 'ff02:', 'ff0x:']
    result = set()
    if 'ValuePoints' in app.udp:
        for vp in app.udp['ValuePoints']:
            for valueSet in vp['ValueSet']:
                if not valueSet:
                    continue
                #print(valueSet)
                valueSetString = str(valueSet)
                valueSetString = valueSetString.lower()
                for address in details:
                    if address in valueSetString:
                        ipExtract = re.findall( r'[0-9]+(?:\.[0-9]+){3}', valueSetString)
                        ipExtract2 = re.findall(r'(([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,7}:|([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2}|([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3}|([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4}|([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))', valueSetString)
                        if len(ipExtract) >0:
                            for c in ipExtract:
                                if '127.0.0.1' in c:
                                    continue
                                if  c.startswith(address):
                                    print(c)
                                    result.add(c)
                        elif len(ipExtract2) > 0:
                            if len(ipExtract2[0]) > 0:
                                result.add(ipExtract2[0][0])

                #for keys in valueSet.keys():
                #    for key in keys:
                #        for value in valueSet[key]:
                #            if addresses.match(value):


    return result



def getStats(appsToAnalyze):
    countAppsLocalAddresses = 0
    countAppsUsingUPNP = 0
    countAppsLocalIpOrUPNP = 0
    countBroadcastAndMulticast = 0

    localAddresses = []
    for app in appsToAnalyze:
        currentLocal = getAllLocalAddresses(app)
        if len(currentLocal) > 0:
            countAppsLocalAddresses = countAppsLocalAddresses + 1

        currentUseUPNP = isAppUsingUPNP(app)
        if currentUseUPNP:
            countAppsUsingUPNP =  countAppsUsingUPNP + 1

        multicast = doesAppContainMulticast(app)
        if len(multicast) > 0:
            countBroadcastAndMulticast = countBroadcastAndMulticast + 1

        if currentUseUPNP or len(currentLocal) > 0 or len(multicast) > 0:
            countAppsLocalIpOrUPNP = countAppsLocalIpOrUPNP + 1



        currentSet = currentLocal.union(multicast)

        localAddresses.append(currentSet)

    return localAddresses, countAppsLocalIpOrUPNP, countAppsUsingUPNP, countAppsLocalAddresses,countBroadcastAndMulticast


In [ ]:
genLocalAddresses, genCountIpUpnp, genCountUPNP, genCountLocal,genCountBroadcast = getStats(genDataset)
%store genLocalAddresses
%store genCountIpUpnp
%store genCountUPNP
%store genCountLocal
%store genCountBroadcast

In [ ]:
verifiedLocalAddresses, verifiedCountIpUpnp, verifiedCountUPNP, verifiedCountLocal, verifiedCountBroadcast = getStats(verified_dataset)
%store verifiedLocalAddresses
%store verifiedCountIpUpnp
%store verifiedCountUPNP
%store verifiedCountLocal
%store verifiedCountBroadcast


In [ ]:


def getLocalAddressCount(localAddresesFromApps):
    result = {}
    for app in localAddresesFromApps:
        for l in app:
            result[l] = result.get(l, 0) + 1
    return result

In [ ]:
%store -r genLocalAddresses
%store -r genCountIpUpnp
%store -r genCountUPNP
%store -r genCountLocal
%store -r genCountBroadcast


In [ ]:
%store -r verifiedLocalAddresses
%store  -r verifiedCountIpUpnp
%store -r  verifiedCountUPNP
%store -r  verifiedCountLocal
%store -r  verifiedCountBroadcast

In [ ]:
verifiedLocalAddresses

In [ ]:

genCount = getLocalAddressCount(genLocalAddresses)
genCount['upnp'] = genCountUPNP


verCount = getLocalAddressCount(verifiedLocalAddresses)
verCount['upnp'] = verifiedCountUPNP

In [ ]:
print(janCount)

In [ ]:
def addAll(result, toAdd):
    for x in toAdd:
        result.add(x)
    return result

allKeys = set()
allKeys =addAll(allKeys, genCount.keys())
allKeys =addAll(allKeys, verCount.keys())

In [ ]:
import pandas as pd
data = []

for key in allKeys:
    row = []
    row.append(key)
    row.append(genCount.get(key, 0))
    row.append(verCount.get(key, 0))

    data.append(row)


df = pd.DataFrame(data, columns=['Address', "GP-2022", "IoT-VER"])

In [ ]:
df

In [ ]:
df.to_csv("all.csv", index = False)

In [ ]:
len(genDataset)

In [ ]:
def readAllPermissionFilesVerified(path, name, mapping):
    result = []
    paths = []
    for app_name in os.listdir(f"{path}"):
        permission_data = util.getJsonData(f"{path}/{app_name}")
        app_name = app_name[0:len(app_name) - 5]
        if app_name in mapping:
            if mapping[app_name] == name:
                result.append(permission_data)
                paths.append(f"{path}/{app_name}")
        else:
            result.append(permission_data)
            paths.append(f"{path}/{app_name}")


    return result, paths

In [ ]:
def readAllPermissionFiles(path):
    result = []
    for app in os.listdir(f"{path}"):
        result.append(util.getJsonData(f"{path}/{app}"))


    return result

In [ ]:
def countPermissions(dataset):
    result = {}
    for app in dataset:
        for permission in app:
            result[permission] = result.get(permission, 0) + 1

    return result

In [ ]:
genDatasetPermission = readAllPermissionFiles("//permission/gp_2022/permissions/")


In [ ]:
mapping_json = {}
with open("//verified_dataset/result.json", "r") as f:
    mapping_json = json.load(f)

In [ ]:
neupaneDatasetPermission, neupane_permission_names = readAllPermissionFilesVerified("/permission/neupane/permissions/", "neupane", mapping_json)
iotspotterDatasetPermission, iotspotter_permission_names = readAllPermissionFilesVerified("/permission/iotspotter/permissions/", "iotspotter", mapping_json)
iotprofilerDatasetPermission, iotprofiler_permission_names = readAllPermissionFilesVerified("/permission/iotprofiler/permissions/", "iotprofiler", mapping_json)

In [ ]:
verDatasetPermission = neupaneDatasetPermission + iotspotterDatasetPermission + iotprofilerDatasetPermission
ver_permission_names = neupane_permission_names + iotspotter_permission_names + iotprofiler_permission_names

In [ ]:
allPermissionGen = countPermissions(genDatasetPermission)

allPermissionVER = countPermissions(verDatasetPermission)

In [ ]:
permissionsToLookFor = ["android.permission.BLUETOOTH", "android.permission.BLUETOOTH_ADMIN", "android.permission.BLUETOOTH_ADVERTISE", "android.permission.BLUETOOTH_CONNECT", "android.permission.BLUETOOTH_SCAN", "android.permission.INTERNET", "android.permission.NFC", "android.permission.NFC_PREFERRED_PAYMENT_INFO", "android.permission.NFC_TRANSACTION_EVENT"]

In [ ]:
%store allPermissionGen
%store allPermissionVER

In [ ]:
blPermission = ["android.permission.BLUETOOTH", "android.permission.BLUETOOTH_ADMIN", "android.permission.BLUETOOTH_ADVERTISE", "android.permission.BLUETOOTH_CONNECT", "android.permission.BLUETOOTH_SCAN"]
def hasBl(dataset):
    count = 0
    for app in dataset:
        for permission in app:
            if permission in blPermission:
                count = count + 1
                break
    return count

In [ ]:
formatted_df = format_table(df)

In [ ]:
formatted_df = formatted_df.sort_values(by=['Address'])

In [ ]:
df.to_csv('localAddresses-save_ccs.csv', index=False)

In [ ]:

%store -r allPermissionGen
%store -r allPermissionVER

In [ ]:
print(formatted_df.to_latex(index=False))


In [ ]:
len(verifiedLocalAddresses)

In [ ]:
len(genLocalAddresses)

In [ ]:
genCountIpUpnp

In [ ]:
verifiedCountIpUpnp

In [ ]:
genCountLocal

In [ ]:
verifiedCountLocal

In [ ]:
genCountBroadcast

In [ ]:
verifiedCountBroadcast

In [ ]:
countFromUIJan = 0
appsWithUI = []
for i in range(0,len(janLocalAddresses)):
    x = janLocalAddresses[i]
    if 'fromui.local' in x:
        countFromUIJan = countFromUIJan + 1
        appsWithUI.append(janDataset[i])

In [ ]:
countFromUIJan

In [ ]:
import random

for i in range(0,len(appsWithUI)):
    print(appsWithUI[random.randint(0,len(appsWithUI)-1)].amqp)

In [ ]:
verifiedCountLocal/9889*100

In [ ]:
genCountLocal/949*100

In [ ]:
verifiedCountLocal

In [ ]:
verifiedCountLocal

In [ ]:
countFromUIJan = 0
appsWithUI = []
for i in range(0,len(janLocalAddresses)):
    x = janLocalAddresses[i]
    if 'fromui.local' in x:
        countFromUIJan = countFromUIJan + 1
        appsWithUI.append(janDataset[i])

In [ ]:
appsWithUpnp = []
for i in range(0,len(janDataset)):
    x = isAppUsingUPNP(janDataset[i])
    if x:
        appsWithUpnp.append(janDataset[i])

In [ ]:
len(appsWithUpnp)

In [ ]:
import random
randomUPNP = set()
while len(randomUPNP) < 15:
    randomUPNP.append(appsWithUpnp[random.randint(0,len(appsWithUpnp)-1)].amqp["pname"])

In [ ]:
randomUPNP

In [ ]:
import random

In [ ]:
appsWithMulticast = []
for i in range(0,len(verified_dataset)):
    x = doesAppContainMulticast(verified_dataset[i])
    if x:
        appsWithMulticast.append(verified_dataset[i])

In [ ]:
randomMulticast = set()
while len(randomMulticast) < 10:
    randomMulticast.add(appsWithMulticast[random.randint(0,len(appsWithMulticast)-1)].amqp["pname"])

In [ ]:
def doesAppHaveBL(app):
    for permission in app:
        if permission in blPermission:
            return True
    return False

In [ ]:
appsWithBL = []
for i in range(0,len(verDatasetPermission)):
    x = doesAppHaveBL(verDatasetPermission[i])
    if x:
        appsWithBL.append(ver_permission_names[i])

In [ ]:
randomBL = set()
while len(randomBL) < 10:
    randomBL.add(appsWithBL[random.randint(0,len(appsWithBL)-1)])

In [ ]:
appsWithUI = []
apps_with_local_addresses = []
for i in range(0,len(verifiedLocalAddresses)):
    x = verifiedLocalAddresses[i]
    if 'fromui.local' in x:
        appsWithUI.append(verified_dataset[i])

    else:
        apps_with_local_addresses.append(verified_dataset[i])



In [ ]:
randomUI = set()
while len(randomUI) < 10:
    randomUI.add(appsWithUI[random.randint(0,len(appsWithUI)-1)].amqp["pname"])

In [ ]:
randomLocal = set()
while len(randomLocal) < 10:
    randomLocal.add(apps_with_local_addresses[random.randint(0,len(apps_with_local_addresses)-1)].amqp["pname"])

In [ ]:
for app in randomMulticast:
    print(app)

In [ ]:
for app in randomBL:
    print(app)

In [ ]:
for app in randomUI:
    print(app)

In [ ]:
for app in randomLocal:
    print(app)

In [ ]:
verifiedLocalAddresses

In [ ]:
def get_category_count_broadcast_multicast(dataset_addresses, addresses, except_addresses=[]):
    count = 0
    for app in dataset_addresses:
        found = False
        for add in app:
            for look_for in addresses:
                if add.startswith(look_for):
                    tmp = True
                    for not_address in except_addresses:
                        if add.startswith(not_address):
                            tmp = False

                    if tmp: #avoid case where it previously found was already true
                        found = True


        if found:
            count += 1
    return count

In [ ]:
#    count = 0
#    for app in dataset_addresses:
#        found = False
#        for add in app:
#            if add.startswith(address) and add not in except_addresses:
#                found = True


#        if found:
#            count += 1
#    return count



def get_category_count(dataset_addresses, address, except_addresses=[]):
    if type(address) == type(""):
        address = [address]

    return get_category_count_broadcast_multicast(dataset_addresses, address, except_addresses)



def get_row(dataset1, dataset2, address, except_address = []):
    return [get_category_count(dataset1, address, except_address), get_category_count(dataset2, address, except_address)]


In [ ]:
rows = []
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "10.", []) )

rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "10.0.0.172", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "10.0.0.200", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "10.0.2.2", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "10.", ["10.0.0.172", "10.0.0.200", "10.10.2.2"]))



rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "172.", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "172.17.0.1", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "172.", ["172.17.0.1"]) )




rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.", []) )

rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.0.1", []) )
#rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.0.", ["192.168.0.1"]))
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.1.1", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.1.3", []) )
#rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.1.", ["192.168.1.1", "192.168.1.3"]))
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.4.1", []) )

# fixme add other
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "192.168.", ["192.168.0.1", "192.168.1.1", "192.168.4.1," ]) )


rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "fe80:", []) )

# broadcast multicast issue
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, ["2", "f"], ["fe80:", "fromui.local"]) )

rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "224.0.0.251", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "239.255.255.250", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "255.255.255.255", []) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "2", ["224.0.0.251", "239.255.255.250","255.255.255.255" ]) )
rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "f", ["fe80:", "fromui.local"]) )

rows.append(get_row(verifiedLocalAddresses, genLocalAddresses, "fromui.local", []) )


rows.append([hasBl(verDatasetPermission), hasBl(genDatasetPermission)])


In [ ]:
rows

In [ ]:
rows

In [ ]:
import pandas as pd


df_iot_gp = pd.DataFrame(rows, index= ["10.*.*.*", "10.0.0.172", "10.0.0.200", "10.10.2.2", "other", "172.16-31.*","172.17.0.1", "other" , "192.168.*.*", "192.168.0.1", "192.168.1.1", "192.168.1.3", "192.168.4.1", "other", "fe80", "Broad-\Multicast", "224.0.0.251", "239.255.255.250", "255.255.255.255", "IPv4 other", "IPv6 other", "fromUI.local", "Bluetooth"  ], columns = ["IoT-VER", "GP-2022"])

In [ ]:
def format_table_iot_gp(table):
    gp_2022 = []
    iot_ver = []
    address = []
    for row in table.iterrows():
        address.append(row[0])
        iot_ver.append("{} ({:.2f}%)".format(row[1]['IoT-VER'],row[1]['IoT-VER']/9889*100))
        gp_2022.append("{} ({:.2f}%)".format(row[1]['GP-2022'],row[1]['GP-2022']/947*100))

    return pd.DataFrame({"Address": address, "IoT-VER": iot_ver , "GP-2022": gp_2022})


In [ ]:
formatted = format_table_iot_gp(df_iot_gp)

In [ ]:
formatted

In [ ]:
print(formatted.to_latex(index=False))

In [ ]:
verCount = getLocalAddressCount(verifiedLocalAddresses)


In [ ]:
verCount

In [ ]:
dict(sorted(verCount.items(), key=lambda item: item[1], reverse=True))
